In [1]:
from gurobipy import *
import numpy as np

In [2]:
def ri(low,high,m_one=0.):
    p = np.random.uniform(0,1)
    if(m_one < p):
        return np.random.randint(low,high)
    return -1

In [3]:
opt_mod = Model(name = "vessel swap body routing")

Restricted license - for non-production use only - expires 2023-10-25


In [4]:
speed = 333
coupling_time = 15
decoupling_time = 15
body_capacity = 50
max_bodies = 3
n_nodes = 5
n_vessels = 3
n_bodies = 5
n_requests = 5
unscheduled_cost_coeff = 10000
service_time = 2

In [5]:
distances = np.array([[0,64100,64700,59000,68800],
                     [64100,0,10900,6200,6800],
                     [64700,10900,0,5800,4100],
                     [59000,6200,5800,0,9800],
                     [68800,6800,4100,9800,0]
                     ])

In [6]:
Nodes = tuplelist()
for i in range(n_nodes):
    Nodes.append((i,1))
Vessels = tuplelist()
for i in range(n_vessels):
    Vessels.append((i,ri(0,n_nodes),0))
Bodies = tuplelist()
for i in range(n_bodies):
    vessel = ri(0,n_vessels)
    Bodies.append((i,vessel,Vessels[vessel][1]))
Requests = tuplelist()
for i in range(n_requests):
    at_body_id = ri(0,n_bodies,.7)
    pick_id    = -1 if at_body_id != -1 else ri(0,n_nodes)
    pick_from  = -1 if at_body_id != -1 else ri(0,21600)
    pick_to    = -1 if at_body_id != -1 else pick_from + ri(0,5760)    
    deliv_id   = ri(0,n_nodes)
    deliv_from = ri(0,21600) if at_body_id != -1 else pick_to + ri(7200,14400)
    deliv_to   = deliv_from + ri(0,5760) if at_body_id != -1 else deliv_from + ri(0,5760)
    num_containters = ri(0,body_capacity/2)
    Requests.append((i,pick_id,pick_from,pick_to,deliv_id,deliv_from,deliv_to,num_containters,at_body_id))
Requests

<gurobi.tuplelist (5 tuples, 9 values each):
 ( 0 , -1 , -1    , -1    , 2 , 21098 , 24121 , 1  , 0  )
 ( 1 , -1 , -1    , -1    , 1 , 1026  , 1427  , 23 , 1  )
 ( 2 , 3  , 16873 , 18616 , 3 , 30283 , 34557 , 21 , -1 )
 ( 3 , 4  , 15297 , 20059 , 1 , 31832 , 37492 , 16 , -1 )
 ( 4 , 4  , 17988 , 18907 , 3 , 30443 , 32175 , 10 , -1 )
>

In [15]:
nodes_requests  = {}
nodes_locations = {}
Pickup_nodes        = []
Pickup_nodes_copy   = []
Delivery_nodes      = []
Delivery_nodes_copy = []
for i in range(n_requests):
    if(Requests[i][1] != -1):
        label = "P_" + str(i)
        Pickup_nodes.append(label)
        nodes_requests[label] = i
        nodes_locations[label] = Requests[i][1]
        
        label_copy = "P'_" + str(i + n_requests)
        Pickup_nodes_copy.append(label_copy)
        nodes_requests[label_copy] = i
        nodes_locations[label_copy] = Requests[i][1]
    
    if(Requests[i][4] != -1):
        label = "D_" + str(i + 2*n_requests)
        Delivery_nodes.append(label)
        nodes_requests[label] = i
        nodes_locations[label] = Requests[i][4]
        
        label_copy = "D'_" + str(i + 3*n_requests)
        Delivery_nodes_copy.append(label_copy)
        nodes_requests[label_copy] = i
        nodes_locations[label_copy] = Requests[i][4]
        
I = Pickup_nodes + Pickup_nodes_copy + Delivery_nodes + Delivery_nodes_copy

J_0 = []
for i in range(n_vessels):
    label = "o_v_" + str(i)
    J_0.append(label)
    nodes_locations[label] = Vessels[i][1]
for i in range(n_bodies):
    label = "o_b_" + str(i)
    J_0.append(label)
    nodes_locations[label] = Bodies[i][2]
    
Dummy = ["*"]
nodes_locations["*"] = -1

N = J_0 + I + Dummy
print(N)
print(nodes_requests)
print(nodes_locations)

['o_v_0', 'o_v_1', 'o_v_2', 'o_b_0', 'o_b_1', 'o_b_2', 'o_b_3', 'o_b_4', 'P_2', 'P_3', 'P_4', "P'_7", "P'_8", "P'_9", 'D_10', 'D_11', 'D_12', 'D_13', 'D_14', "D'_15", "D'_16", "D'_17", "D'_18", "D'_19", '*']
{'D_10': 0, "D'_15": 0, 'D_11': 1, "D'_16": 1, 'P_2': 2, "P'_7": 2, 'D_12': 2, "D'_17": 2, 'P_3': 3, "P'_8": 3, 'D_13': 3, "D'_18": 3, 'P_4': 4, "P'_9": 4, 'D_14': 4, "D'_19": 4}
{'D_10': 2, "D'_15": 2, 'D_11': 1, "D'_16": 1, 'P_2': 3, "P'_7": 3, 'D_12': 3, "D'_17": 3, 'P_3': 4, "P'_8": 4, 'D_13': 1, "D'_18": 1, 'P_4': 4, "P'_9": 4, 'D_14': 3, "D'_19": 3, 'o_v_0': 2, 'o_v_1': 0, 'o_v_2': 4, 'o_b_0': 4, 'o_b_1': 4, 'o_b_2': 0, 'o_b_3': 0, 'o_b_4': 4, '*': -1}


In [43]:
A_1 = tuplelist()
A_2 = tuplelist()
A_3 = tuplelist()
A_4 = tuplelist()
A_5 = tuplelist()
A_sink = tuplelist()
A_D = tuplelist()
A = tuplelist()

# A_1
for o in J_0:
    for n in N:
        if(o == n):
            continue
        A_1.append((o,n))
###################################################################
# A_2
for p,p_copy in zip(Pickup_nodes,Pickup_nodes_copy):
    A_2.append((p,p_copy))
for p in Pickup_nodes:
    r = nodes_requests[p]
    for n in N:
        if(n[0] == 'P' or n[0] == 'D'):
            if(nodes_requests[n] == r):
                continue
        A_2.append((p,n))
###################################################################
# A_3
for p_copy,d in zip(Pickup_nodes_copy,Delivery_nodes):
    A_3.append((p_copy,d))
for p_copy in Pickup_nodes_copy:
    r = nodes_requests[p_copy]
    for n in N:
        if(n[0] == 'P' or n[0] == 'D'):
            if(nodes_requests[n] == r):
                continue
        A_3.append((p_copy,n))
###################################################################
# A_4
for d,d_copy in zip(Delivery_nodes,Delivery_nodes_copy):
    A_4.append((d,d_copy))
for d in Delivery_nodes:
    r = nodes_requests[d]
    for n in N:
        if(n[0] == 'P' or n[0] == 'D'):
            if(nodes_requests[n] == r):
                continue
        A_4.append((d,n))
###################################################################
# A_5
for d_copy in Delivery_nodes_copy:
    r = nodes_requests[d_copy]
    for n in N:
        if(n[0] == 'P' or n[0] == 'D'):
            if(nodes_requests[n] == r):
                continue
        A_5.append((d_copy,n))
###################################################################
# A_sink
for n in J_0 + I:
    A_sink.append((n,Dummy[0]))

A = A_1 + A_2 + A_3 + A_4 + A_5 + A_sink

# A_D
for i,j in A:
    if(nodes_locations[i] == nodes_locations[j]):
        A_D.append((i,j))
A_D += A_sink

In [17]:
travel_time = {}
for a in A:
    dep = a[0]
    arr = a[1]
    
    time = distances[nodes_locations[dep],nodes_locations[arr]] / speed if nodes_locations[arr] != -1 else 0
    
    if(dep[:2] == "P_" and arr[:3] == "P'_"):
        if(nodes_requests[dep] == nodes_requests[arr]):
            r = nodes_requests[dep]
            time = coupling_time * Requests[r][7]
            
    if(dep[:2] == "D_" and arr[:3] == "D'_"):
        if(nodes_requests[dep] == nodes_requests[arr]):
            r = nodes_requests[dep]
            time = decoupling_time * Requests[r][7]
            
    travel_time[a] = time

In [18]:
z_v      = []
beta_b   = []
omega_bv = []
x_br     = []
delta_b  = []
gamma_r  = []
tau_a_v  = []
tau_d_v  = []
tau_a_b  = []
tau_d_b  = []
y_v      = []
phi_b    = []

for v in range(n_vessels):
    z_v.append({})
    for arc in A:
        z_v[v][arc] = opt_mod.addVar(name = "z_" + str(v) + "_(" + arc[0] + "," + arc[1] + ")", vtype = GRB.BINARY)

for b in range(n_bodies):
    beta_b.append({})
    for arc in A:
        beta_b[b][arc] = opt_mod.addVar(name = "beta_" + str(b) + "_(" + arc[0] + "," + arc[1] + ")", vtype = GRB.BINARY)

for v in range(n_vessels):
    omega_bv.append([])
    for b in range(n_bodies):
        omega_bv[v].append({})
        for arc in A:
            omega_bv[v][b][arc] = opt_mod.addVar(name = "omega_" + str(b) + "_" + str(v) + "_(" + arc[0] + "," + arc[1] + ")", vtype = GRB.BINARY)

for r in range(n_requests):
    x_br.append([])
    for b in range(n_bodies):
        x_br[r].append({})
        for arc in A:
            x_br[r][b][arc] = opt_mod.addVar(name = "x_br_" + str(b) + "_" + str(r) + "_(" + arc[0] + "," + arc[1] + ")", vtype = GRB.BINARY)

for b in range(n_vessels):
    delta_b.append({})
    for arc in A:
        delta_b[b][arc] = opt_mod.addVar(name = "delta_" + str(b) + "_(" + arc[0] + "," + arc[1] + ")", vtype = GRB.BINARY)

for r in range(n_requests):
    gamma_r.append(opt_mod.addVar(name = "gamma_" + str(r), vtype = GRB.BINARY))

for v in range(n_vessels):
    tau_a_v.append([])
    for n in N:
        tau_a_v[v].append(opt_mod.addVar(name = "tau_a_" + str(v) + "_" + n, vtype = GRB.CONTINUOUS, lb = 0))

for v in range(n_vessels):
    tau_d_v.append([])
    for n in N:
        tau_d_v[v].append(opt_mod.addVar(name = "tau_d_" + str(v) + "_" + n, vtype = GRB.CONTINUOUS, lb = 0))

for b in range(n_bodies):
    tau_a_b.append([])
    for n in N:
        tau_a_b[b].append(opt_mod.addVar(name = "tau_a_" + str(b) + "_" + n, vtype = GRB.CONTINUOUS, lb = 0))

for b in range(n_bodies):
    tau_d_b.append([])
    for n in N:
        tau_d_b[b].append(opt_mod.addVar(name = "tau_d_" + str(b) + "_" + n, vtype = GRB.CONTINUOUS, lb = 0))

for v in range(n_vessels):
    y_v.append({})
    for i in N:
        for j in N:
            if(i == j):
                continue
            y_v[v][(i,j)] = opt_mod.addVar(name = "y_" + str(v) + "_(" + i + "," + j + ")", vtype = GRB.BINARY)

for b in range(n_bodies):
    phi_b.append({})
    for i in N:
        for j in N:
            if(i == j):
                continue
            phi_b[b][(i,j)] = opt_mod.addVar(name = "phi_" + str(b) + "_(" + i + "," + j + ")", vtype = GRB.BINARY)

opt_mod.update()

In [11]:
obj_fn = quicksum(quicksum(travel_time[a]*z_v[v][a] for a in A) for v in range(n_vessels)) + quicksum(unscheduled_cost_coeff * (1 - gamma_r[r]) for r in range(n_requests))
opt_mod.setObjective(obj_fn,GRB.MINIMIZE)

In [45]:
c_2 = []
c_3 = []
c_4 = {}

for v in range(n_vessels):
    c_2.append(
        opt_mod.addConstr(
            quicksum(
                z_v[v][arc] for arc in A.select(J_0[v],"*")) == 1), name = "c_2")
    
for v in range(n_vessels):
    c_3.append(
        opt_mod.addConstr(
            quicksum(
                z_v[v][arc] for arc in A_sink) == 1), name = "c_3")

for v in range(n_vessels):
    for i in I:
        c_4[(v,i)] = opt_mod.addConstr(
                                        quicksum(z_v[v][arc] for arc in A.select("*",i)) == quicksum(z_v[v][arc] for arc in A.select(i,"*")), name = "c_4")
        
c_5 = []
c_6 = []
c_7 = {}

for b in range(n_bodies):
    c_5.append(
        opt_mod.addConstr(
            quicksum(
                beta_b[b][arc] for arc in A.select(J_0[n_vessels + b],"*")) == 1), name = "c_5")
    
for b in range(n_bodies):
    c_6.append(
        opt_mod.addConstr(
            quicksum(
                beta_b[b][arc] for arc in A_sink) == 1), name = "c_6")

for b in range(n_bodies):
    for i in I:
        c_7[(b,i)] = opt_mod.addConstr(
                                        quicksum(beta_b[b][arc] for arc in A.select("*",i)) == quicksum(beta_b[b][arc] for arc in A.select(i,"*")), name = "c_7")

c_8 = {}
c_9 = {}

for b in range(n_bodies):
    for arc in A_D:
        c_8[(b,arc)] = opt_mod.addConstr(
                                        beta_b[b][arc] = delta[b][arc] + quicksum(omega[v][b][arc] for v in range(n_vessels)), name = "c_8")

for b in range(n_bodies):
    for arc in set(A).difference(A_D):
        c_9[(b,arc)] = opt_mod.addConstr(
                                        beta_b[b][arc] = quicksum(omega[v][b][arc] for v in range(n_vessels)), name = "c_9")